In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import DatetimeTickFormatter, HoverTool
from bokeh.models.widgets import CheckboxGroup

In [ ]:
df = pd.read_csv("clean EEG2.csv")

if("Unnamed: 0" in df.columns):
    df = df.drop(columns=['Unnamed: 0'])

df.head(30)


In [ ]:
df1 = df[df.isna().any(axis=1)]
print(len(df))
print(len(df1))

df1.head(30)


In [ ]:
#Split data frames into big gaps
df = df.sort_values(by=['time'])
df.index = range(len(df))
 
pastTime = df["time"][0]
currentPeriod = 1
for i in range(len(df)):
    if pastTime < (df["time"][i] - .1*60*1000):
        currentPeriod += 1
    pastTime = df["time"][i]/1000
    df.at[i,"period"] = currentPeriod
    df.at[i, "rTime"] = datetime.utcfromtimestamp(pastTime.astype(int))
    df.at[i, "clock"] = df.at[i,"rTime"].time()


In [ ]:
p = figure(plot_width = 1000, plot_height = 1000, 
           title = 'EEG',
           x_axis_label = 'Time', y_axis_label = 'Nano volts')
p.xaxis.formatter=DatetimeTickFormatter(days="%m/%d %H:%M",
months="%m/%d %H:%M",
hours="%m/%d %H:%M",
minutes="%m/%d %H:%M")

hover = HoverTool()
hover.tooltips = [('hour:minute', '@rTime{%H:%M}'), ('second:milisecnd', '@rTime{%S.%3Ns}') , ('volts', "$y"), ('index', '$index'), ("ID", "@ID")]
hover.formatters = {'@rTime': 'datetime'}
p.add_tools(hover)

# Create a blank figure with labels

p.line( 'clock','col1',  color='skyblue', source = df, legend_label="channel1")
p.line( 'clock','col2',  color='red', source = df, legend_label="channel2")
p.line( 'clock','col3', color='green', source = df, legend_label="channel3")

p.add_tools(hover)
p.legend.location = "top_left"
p.legend.click_policy="hide"

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)


In [ ]:
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_file, show

p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.title.text = 'Click on legend entries to hide the corresponding lines'


p.line(df['rTime'], df['col1'], line_width=2,color='skyblue',  alpha=0.8, legend_label="channel1")
p.line(df['rTime'], df['col2'], line_width=2, color='green',  alpha=0.8, legend_label="channel2")
p.line(df['rTime'], df['col3'], line_width=2, color='red',  alpha=0.8, legend_label="channel3")

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_notebook()

show(p)